### Iterating Over Rows

In [51]:
import pandas as pd

# Loading the Dataset
df = pd.read_csv("file.csv", delimiter="\t")
# Converting date Column to Datetime Format
df["date"] = df["date"].map(lambda date: pd.to_datetime(date))
# Creating the Result Dataset
result = pd.DataFrame(columns=["user_id", "start_date", "end_date", "log", "length"])
# Iterating over Dataset Rows to Create the Result Dataset
log0 = -1
user_id0 = -1
for i, row in df.iterrows():
    date, user_id, log = row
    # Tracking Change in log or user_id
    if log != log0 or user_id != user_id0:
        # Creating a New Row
        new_row = pd.DataFrame([{
            "user_id": user_id,
            "start_date": date,
            "end_date": date,
            "log": log,
            "length": 1}])
        result = pd.concat([result, new_row], ignore_index=True)
    else:
        # Updating end_date and length Columns
        result.iloc[-1, result.columns.get_loc("end_date")] = date
        result.iloc[-1, result.columns.get_loc("length")] = (date - result.iloc[-1]["start_date"]).days + 1
    log0 = log
    user_id0 = user_id
# Sorting Values by start_date and user_id
result = result.sort_values(["start_date", "user_id"])
# Reindexing the Result Dataset
result.index = range(1, result.shape[0] + 1)
# Displaying the Result Dataset
result

C:\Users\PQ\AppData\Local\Temp\ipykernel_10672\4113241328.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, new_row], ignore_index=True)


,user_id,start_date,end_date,log,length
1,51725,2019-01-01,2019-01-01,1,1
2,116054,2019-01-01,2019-01-01,0,1
3,140223,2019-01-01,2019-01-02,1,2
4,275100,2019-01-01,2019-01-02,0,2
5,313779,2019-01-01,2019-01-03,1,3
...,...,...,...,...,...
889,524607,2019-03-31,2019-03-31,0,1
890,652373,2019-03-31,2019-03-31,1,1
891,668095,2019-03-31,2019-03-31,0,1
892,733297,2019-03-31,2019-03-31,1,1


### Using Pandas Methods

In [52]:
import pandas as pd
import datetime as dt

# Loading the Dataset
df = pd.read_csv("file.csv", delimiter="\t")
# Converting date Column to Datetime Format
df["date"] = df["date"].map(lambda date: pd.to_datetime(date))
# Sorting Values by user_id and date
df = df.sort_values(["user_id", "date"])
# Tracking Change in log or user_id
df["log_status"] = (df["log"] != df["log"].shift())
df["user_id_status"] = (df["user_id"] != df["user_id"].shift())
df["status"] = df["log_status"] + df["user_id_status"]
# Filtering Related Rows and Dropping Extra Columns
df.drop(["log_status", "user_id_status"], axis=1, inplace=True)
df = df[df["status"]].drop("status", axis=1)
# Renaming date Column to start_date
df.rename(columns={"date": "start_date"}, inplace=True)
# Defining end_date and length Columns
df["end_date"] = df["start_date"].shift(-1) - dt.timedelta(1)
df["end_date"] = df.apply(lambda row: row["end_date"] if row["end_date"] >= row["start_date"] else dt.datetime(2019,3,31), axis=1)
df["length"] = df["end_date"] - df["start_date"]
df["length"] = df["length"].map(lambda date: date.days + 1)
# Reordering Dataset's Columns
df = df[["user_id", "start_date", "end_date", "log", "length"]]
# Sorting Values by start_date and user_id
df.sort_values(["start_date", "user_id"], inplace=True)
# Reindexing the Dataset
df.index = range(1, df.shape[0] + 1)
# Saving Final Dataset to Excel
df.to_csv("final.csv", index=False)
# Displaying the Dataset
df

,user_id,start_date,end_date,log,length
1,51725,2019-01-01,2019-01-01,1,1
2,116054,2019-01-01,2019-01-01,0,1
3,140223,2019-01-01,2019-01-02,1,2
4,275100,2019-01-01,2019-01-02,0,2
5,313779,2019-01-01,2019-01-03,1,3
...,...,...,...,...,...
889,524607,2019-03-31,2019-03-31,0,1
890,652373,2019-03-31,2019-03-31,1,1
891,668095,2019-03-31,2019-03-31,0,1
892,733297,2019-03-31,2019-03-31,1,1


### Using SQL Nested Queries

In [53]:
import sqlite3
import numpy as np
import pandas as pd

# Connecting to sqlite3
connector = sqlite3.connect("dkcup_test.db")
cursor = connector.cursor()
# Executing the Related Query
cursor.execute("""
    SELECT
        user_id,
        start_date,
        end_date,
        log,
        CAST((JULIANDAY(end_date) - JULIANDAY(start_date) + 1) AS INTEGER) AS length
    FROM
        (SELECT
            user_id,
            date AS start_date,
            CASE
                WHEN LEAD(date) OVER (ORDER BY user_id) > DATE
                THEN DATE(LEAD(date) OVER (ORDER BY user_id), "-1 DAY")
                ELSE "2019-03-31"
            END AS end_date,
            log
        FROM
            (SELECT 
                *
            FROM
                (SELECT 
                    user_id,
                    date,
                    log,
                    CASE
                        WHEN 
                            LAG(log) OVER (ORDER BY user_id, date) IS NOT log
                                OR LAG(user_id) OVER(ORDER BY user_id, date) != user_id
                        THEN 1
                        ELSE 0 
                    END AS status
                FROM
                    users_log)
            WHERE 
                status = 1)
        ORDER BY date, user_id);
    """)
# Extracting Rows and Columns
rows = np.array(cursor.fetchall()).T
columns = [description[0] for description in cursor.description]
# Closing the Connection to the Database
cursor.close()
connector.close()
# Creating The DataFrame with Rows and Columns
df = pd.DataFrame({
    columns[i]: [rows[i, j] for j in range(rows.shape[1])] for i in range(rows.shape[0])
    })
# Reindexing the Dataset
df.index = range(1, df.shape[0] + 1)
# Displaying the Dataset
df

,user_id,start_date,end_date,log,length
1,51725,2019-01-01,2019-01-01,1,1
2,116054,2019-01-01,2019-01-01,0,1
3,140223,2019-01-01,2019-01-02,1,2
4,275100,2019-01-01,2019-01-02,0,2
5,313779,2019-01-01,2019-01-03,1,3
...,...,...,...,...,...
889,524607,2019-03-31,2019-03-31,0,1
890,652373,2019-03-31,2019-03-31,1,1
891,668095,2019-03-31,2019-03-31,0,1
892,733297,2019-03-31,2019-03-31,1,1


### Using SQL With Statement

In [54]:
import sqlite3
import numpy as np
import pandas as pd

# Connecting to sqlite3
connector = sqlite3.connect("dkcup_test.db")
cursor = connector.cursor()
# Executing the Related Query
cursor.execute("""
    WITH log_status AS (
        SELECT 
            user_id,
            date,
            log,
            CASE
                WHEN 
                    LAG(log) OVER (ORDER BY user_id, date) IS NOT log
                        OR LAG(user_id) OVER(ORDER BY user_id, date) != user_id
                THEN 1
                ELSE 0 
            END AS status
        FROM
            users_log
    ),
    filtered_logs AS (
        SELECT 
            *
        FROM
            log_status
        WHERE status = 1
    ),
    report_format AS (
        SELECT
            user_id,
            date AS start_date,
            CASE
                WHEN LEAD(date) OVER (ORDER BY user_id) > DATE
                THEN DATE(LEAD(date) OVER (ORDER BY user_id), "-1 DAY")
                ELSE "2019-03-31"
            END AS end_date,
            log
        FROM
            filtered_logs
        ORDER BY date, user_id
    )    
    SELECT
        user_id,
        start_date,
        end_date,
        log,
        CAST((JULIANDAY(end_date) - JULIANDAY(start_date) + 1) AS INTEGER) AS length
    FROM
        report_format;
    """)
# Extracting Rows and Columns
rows = np.array(cursor.fetchall()).T
columns = [description[0] for description in cursor.description]
# Closing the Connection to the Database
cursor.close()
connector.close()
# Creating The DataFrame with Rows and Columns
df = pd.DataFrame({
    columns[i]: [rows[i, j] for j in range(rows.shape[1])] for i in range(rows.shape[0])
    })
# Reindexing the Dataset
df.index = range(1, df.shape[0] + 1)
# Saving Final Dataset to Excel
df.to_csv("final-sql.csv", index=False)
# Displaying the Dataset
df

,user_id,start_date,end_date,log,length
1,51725,2019-01-01,2019-01-01,1,1
2,116054,2019-01-01,2019-01-01,0,1
3,140223,2019-01-01,2019-01-02,1,2
4,275100,2019-01-01,2019-01-02,0,2
5,313779,2019-01-01,2019-01-03,1,3
...,...,...,...,...,...
889,524607,2019-03-31,2019-03-31,0,1
890,652373,2019-03-31,2019-03-31,1,1
891,668095,2019-03-31,2019-03-31,0,1
892,733297,2019-03-31,2019-03-31,1,1
